# Hands-on with CUDA.jl

In this notebook, we'll use [`CUDA.jl`](https://github.com/JuliaGPU/CUDA.jl) to control and program NVIDIA GPU resources. If you have an NVIDIA GPU already (in your laptop or desktop), you can run this notebook locally. If not, you can use a GPU available through [Google Colab](https://colab.research.google.com/). This will allow you to use a GPU *for free* for up to 12 hours of interactive experimentation.

We'll use `CUDA.jl` because:
- NVIDIA GPUs are highly available
- CUDA is (for now) the most mature GPU programming paradigm with many libraries and resources
- Many of the underlying concepts transfer to other systems like AMD ROCm and Intel oneAPI

Let's import `CUDA.jl`:

In [ ]:
using Pkg; Pkg.add("CUDA"); Pkg.add("StaticArrays")
using CUDA, StaticArrays

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


CUDA is actually made up of several components:
- The GPU driver, which controls the graphics card and its interactions with the operating system kernel
- The CUDA runtime, which provides the basic CUDA C extensions, CUDA API, and CUDA compiler
- The CUDA libraries, which provide a "GPUified" implementation of BLAS (CUBLAS), RNG (CURAND), and many other things

`CUDA.jl` has prepackaged [artifacts](https://pkgdocs.julialang.org/v1/artifacts/) for many CUDA versions, Julia versions, and OSes. It will download the appropriate one by default and install it for you. This lets us use the high-level Julia functions for CUDA arrays without worrying about the backend API calls (for now). Once `CUDA.jl` has installed, we can try creating some arrays on the GPU. We do this with `CuArray`, which turns a CPU array into a CUDA array by copying it.

`CUDA.jl` also has some convenience constructors for `CuArray`s, like `CUDA.zeros`, `CUDA.rand`, and `undef` initializers. We can even use methods like `map` and `reduce` on these.

In [ ]:
cu_a = CUDA.rand(Float64, 1024, 1024);
typeof(cu_a)

CuArray{Float64, 2, CUDA.DeviceMemory}

In [ ]:
cu_b = map(x->x^2, cu_a);

These `CuArray`s live in **device memory** (GPU RAM), which means their individual elements are only available **from code executing on the GPU**. We know that the GPU memory is relatively limited compared to CPU memory, so how does `CUDA.jl` handle this? Do we have to explicitly "reap" these arrays ourselves?

`CUDA.jl` extends the Julia garbage collector to work with GPU arrays as well. It manages a memory pool of GPU memory and garbage-collected `CuArray`s are in fact returned to the memory pool (rather than being entirely deallocated) in order to reduce GC pressure. `CUDA.jl` recycles arrays in the pool preferentially to speed up future allocations and keep an eye on how close we are to running out of memory. We can inspect the current state of the memory pool with `CUDA.pool_status()`:

In [ ]:
CUDA.pool_status()

Effective GPU memory usage: 6.74% (1016.938 MiB/14.741 GiB)
Memory pool usage: 33.519 MiB (832.000 MiB reserved)


Let's allocate another device array and see how the pool status changes:

In [ ]:
cu_c = CUDA.rand(Float64, 1024, 1024)
cu_c .+= cu_c' # make cu_c symmetric
CUDA.pool_status()

Effective GPU memory usage: 6.74% (1016.938 MiB/14.741 GiB)
Memory pool usage: 49.519 MiB (832.000 MiB reserved)


We should never need to explicitly deallocate CUDA arrays, but we can reduce GC pressure and help CUDA.jl out by indicating when we are done with an array, using `CUDA.unsafe_free!`. As you can guess from the name, you should **never** refer to the array after you've unsafely freed it!

In [ ]:
CUDA.unsafe_free!(cu_b)
CUDA.pool_status()

Effective GPU memory usage: 6.74% (1016.938 MiB/14.741 GiB)
Memory pool usage: 41.519 MiB (832.000 MiB reserved)


### Tips and tricks for higher level array operations

In general, a `map` or `broadcast` over columns/slices launches multiple independent kernels which is usually slower than launching one larger kernel. For example,
```julia
broadcast(eachcol(a)) do x
    prod(x)
end
```
will usually be slower than `prod(a; dims=2)`.

`CUDA.jl` isn't a tensor compiler and generally implements scalar (single-element) kernels for many operations like `map`. It's up to you to effectively use tools like broadcasting to avoid this.

Broadcast fusion is especially helpful for GPU arrays because it turns multiple kernel launches on the same data into one.

## Using `LinearAlgebra` extensions

Already we can see many of the nice array features we covered yesterday apply just as well to `CuArray`s. Many, but not all. `view`s, for example, are inconsistently supported. We can also make use of many wrapped linear algebra functions optimized for GPU via `CUBLAS`. We'll set the `CUBLAS` logger to prove to ourselves that it is in fact being used from our nice compact call:

In [ ]:
using LinearAlgebra
CUBLAS.cublasLoggerConfigure(1, 0, 1, C_NULL) # normally we don't do this, just for demonstration purposes here
cu_d = cu_c * cu_a;

I! cuBLAS (v12.3) function cublasStatus_t cublasGemmEx(cublasHandle_t, cublasOperation_t, cublasOperation_t, int, int, int, const void*, const void*, cudaDataType, int, const void*, cudaDataType, int, const void*, void*, cudaDataType, int, cublasComputeType_t, cublasGemmAlgo_t) called:
i!  handle: type=cublasHandle_t; val=POINTER (IN HEX:0x0x13145540)
i!  transa: type=cublasOperation_t; val=CUBLAS_OP_N(0)
i!  transb: type=cublasOperation_t; val=CUBLAS_OP_N(0)
i!  m: type=int; val=1024
i!  n: type=int; val=1024
i!  k: type=int; val=1024
i!  alpha: type=void; val=POINTER (IN HEX:0x0x303800400)
i!  A: type=void; val=POINTER (IN HEX:0x0x303000000)
i!  Atype: type=cudaDataType_t; val=CUDA_R_64F(1)
i!  lda: type=int; val=1024
i!  B: type=void; val=POINTER (IN HEX:0x0x302000000)
i!  Btype: type=cudaDataType_t; val=CUDA_R_64F(1)
i!  ldb: type=int; val=1024
i!  beta: type=void; val=POINTER (IN HEX:0x0x303800800)
i!  C: type=void; val=POINTER (IN HEX:0x0x302800000)
i!  Ctype: type=cudaDataType_t

We also have wrappers for some factorizations:

In [ ]:
qr_c = qr(CUDA.rand(Float64, 64, 32));

I! cuBLAS (v12.3) function cublasStatus_t cublasDgemm_v2_64(cublasHandle_t, cublasOperation_t, cublasOperation_t, int64_t, int64_t, int64_t, const double*, const double*, int64_t, const double*, int64_t, const double*, double*, int64_t) called:
i!  handle: type=cublasHandle_t; val=POINTER (IN HEX:0x0x136315a0)
i!  transa: type=cublasOperation_t; val=CUBLAS_OP_C(2)
i!  transb: type=cublasOperation_t; val=CUBLAS_OP_N(0)
i!  m: type=long; val=16
i!  n: type=long; val=16
i!  k: type=long; val=64
i!  alpha: type=double; val=POINTER (IN HEX:0x0x7ae65b7f96a0)
i!  A: type=double; val=POINTER (IN HEX:0x0x30398c400)
i!  lda: type=long; val=64
i!  B: type=double; val=POINTER (IN HEX:0x0x30398e400)
i!  ldb: type=long; val=64
i!  beta: type=double; val=POINTER (IN HEX:0x0x7ae65b7f9698)
i!  C: type=double; val=POINTER (IN HEX:0x0x303848400)
i!  ldc: type=long; val=16
i! Time: 2025-07-14T23:55:46 elapsed from start 2.533333 minutes or 152.000000 seconds
i!Process=701; Thread=135130737855744; GPU=0; H

## Running multiple functions simultaneously

A `1024 x 1024` matrix is decently hefty but not big enough to saturate the throughput of the GPU. If we have many such matrices, we can spawn multiple separate simultaneous executions on the same GPU to speed up our computation, provided we don't run out of memory. `CUDA.jl` has a nice integration with the built-in Julia threading for this.

In [ ]:
n_arrs = 100
cpu_arrays = [rand(Float64, 1024, 1024) for ix_arr in 1:n_arrs]
results = Vector{Float64}(undef, n_arrs)
@sync begin
    for ix_arr in 1:n_arrs
        Threads.@spawn begin
            results[ix_arr] = mapreduce(sin, *, CuArray(cpu_arrays[ix_arr]))
        end
    end
end

Obviously this example is a bit contrived, but using this basic pattern allows Julia and CUDA to work together to interleave memory copies and useful work, making better overall use of your GPU. We wrap everything in the `@sync` block because `Threads.@spawn` returns immediately after creating and launching its task, and `@sync` forces Julia to wait for all the `@spawn`-ed tasks to complete (and thus for all elements of `results` to be populated).

## Important considerations for memory transfers

- Memory copies to/from/across the GPU are **high latency**. Each one takes a relatively long time to kick off. Thus, it's usually better to do **one large copy** (by concatenating arrays, for example) than **many small copies**.
- By default, `CuArray`s live in the GPU memory and can't be accessed elementwise from the CPU. You, as the programmer, explicitly request memory copies, which helps you control and understand when they happen. However, you can use the CUDA [Unified Memory](https://developer.nvidia.com/blog/unified-memory-cuda-beginners/) [from `CUDA.jl`](https://cuda.juliagpu.org/dev/usage/memory/#Unified-memory), which makes the arrays visible from both CPU and GPU. The CUDA driver copies memory back and forth for you as needed. This can be very convenient for the programmer but can lead to degraded performance depending on your memory access pattern.
- If you have a pre-allocated CPU array you'll be copying to and from frequently (a pre-allocated out of loop buffer, for example), you can **pin** the CPU memory using `CUDA.pin`, which can speed up these copies substantially. The very act of pinning is itself time-consuming, so be sure it's worth it (benchmarks!).

## When the builtin wrappers aren't enough: writing our own kernels 🌽

Sometimes the high level Julia constructs just aren't enough. We need to implement our own high-performance functions that will be executed across the massive thread population. Awesome! However, there are some new complications:
- We have to make efficient use of the hardware, which isn't always intuitive
- We have a restricted subset of the programming language available on-GPU
- For many operations it's not obvious how to translate the algorithm to an efficient GPU implementation

A good example of this latter point is creating an [alias table](https://github.com/LilithHafner/AliasTables.jl/) for discrete categorical sampling (this is a common thing to do for sampling state vectors, for example). Many CPU algorithms exist for this purpose, and some GPU ones too, but the underlying approach is quite different because of the GPU's constraints.

A function we will execute in a massively/"embarrassingly" parallel way on the GPU across a large array is called a *kernel*. We can write a restricted subset of Julia within a kernel. `CUDA.jl` and its backend `GPUCompiler.jl` handle translating & compiling this through LLVM to the underlying GPU IR.

The higher-level functions we worked with before are actually implemented behind-the-scenes as CUDA kernels. It's worth spending a little time to understand what is happening when a kernel executes.



### Quick guide to kernel launches and execution

CUDA spawns a *grid* of *blocks* of *threads*, and each thread is an individual worker. Threads are grouped into *warps* of size 32. Each thread in the warp steps through the function instructions **together**. If two threads in the same warp encounter **different** instructions, the warp can no longer execute in parallel and the performance benefit of using the GPU can disappear. This is called **warp divergence** (very *Star Trek*). In general, one warp out of 10,000 experiencing divergence will not hurt things too badly -- but a large fraction of warps experiencing it will.

**Our goal when writing CUDA kernels is to make efficient use of warp-parallelism.**

When we launch the CUDA kernel, we launch it with `(n_blocks_x, n_blocks_y, n_blocks_z)` blocks of `(n_threads_x, n_threads_y, n_threads_z)` threads. Any of these numbers can be 1 -- but you can also create 3D grids if it's more natural for your problem. In general, it's **not** the case that `n_blocks_x * n_blocks_y * n_blocks_z * n_threads_x * n_threads_y * n_threads_z` threads execute in parallel (though this can happen if the sizes are small). Rather, blocks are scheduled by the CUDA driver, so that block 0 may execute, and then block 65, then block 32. **Block execution order is not guaranteed!**

Because the warps are size 32, it is most efficient to make the thread dimensions a power of 2. So, `n_threads_x = 64` would be a good choice, or `n_threads_y = 512`. **In general**, larger thread counts are better (512 threads-per-block is usually more performant than 64) but of course there are some edge cases.

Within a kernel, each thread has access to some information about its personal location in the overall grid. We can query the `threadIdx()` and `blockIdx()` functions to get the thread's `x`, `y`, and `z` thread and block positions, and `blockDim()` to get the number of threads in a block in the `x`, `y`, and `z` directions.

We start by writing a `function` which may take arguments. `CUDA.jl` kernels aren't yet hooked up to standard Julia IO, but we **can** do some basic printing using the `@cuprintln` macro. **CUDA kernels must always return `nothing`.**

In [ ]:
function my_first_cuda_kernel(a)
    t_x = threadIdx().x
    b_x = blockIdx().x
    l_a = length(a)
    @cuprintln("Hi! My thread index is $t_x and my block index is $b_x. The length of my argument is $l_a.")
    return
end

my_first_cuda_kernel (generic function with 3 methods)

Now we *launch* the kernel on the GPU using the `@cuda` macro. We can specify how many threads and blocks to launch with at this time. **Keep in mind that CUDA kernels cannot see arrays in CPU memory, only GPU arrays.**

In [ ]:
cu_a = CUDA.rand(Float32, 512)

@cuda threads=64 blocks=div(length(cu_a), 64) my_first_cuda_kernel(cu_a)

CUDA.HostKernel for my_first_cuda_kernel(CuDeviceVector{Float32, 1})

However, **stack-allocated objects** like scalar integers, floats, or *tuples* (and thus *StaticArrays*) **can** be passed as arguments to CUDA kernels. 😈

In [ ]:
using StaticArrays

static_a = SVector{16, ComplexF64}(rand(ComplexF64) for si in 1:16)
@cuda threads=16 my_first_cuda_kernel(static_a)

Hi! My thread index is 1 and my block index is 8. Wowow Float32 so speedy.
Hi! My thread index is 2 and my block index is 8. Wowow Float32 so speedy.
Hi! My thread index is 3 and my block index is 8. Wowow Float32 so speedy.
Hi! My thread index is 4 and my block index is 8. Wowow Float32 so speedy.
Hi! My thread index is 5 and my block index is 8. Wowow Float32 so speedy.
Hi! My thread index is 6 and my block index is 8. Wowow Float32 so speedy.
Hi! My thread index is 7 and my block index is 8. Wowow Float32 so speedy.
Hi! My thread index is 8 and my block index is 8. Wowow Float32 so speedy.
Hi! My thread index is 9 and my block index is 8. Wowow Float32 so speedy.
Hi! My thread index is 10 and my block index is 8. Wowow Float32 so speedy.
Hi! My thread index is 11 and my block index is 8. Wowow Float32 so speedy.
Hi! My thread index is 12 and my block index is 8. Wowow Float32 so speedy.
Hi! My thread index is 13 and my block index is 8. Wowow Float32 so speedy.
Hi! My thread index i

CUDA.HostKernel for my_first_cuda_kernel(SVector{16, ComplexF64})

Hi! My thread index is 1 and my block index is 1. The length of my argument is 16.
Hi! My thread index is 2 and my block index is 1. The length of my argument is 16.
Hi! My thread index is 3 and my block index is 1. The length of my argument is 16.
Hi! My thread index is 4 and my block index is 1. The length of my argument is 16.
Hi! My thread index is 5 and my block index is 1. The length of my argument is 16.
Hi! My thread index is 6 and my block index is 1. The length of my argument is 16.
Hi! My thread index is 7 and my block index is 1. The length of my argument is 16.
Hi! My thread index is 8 and my block index is 1. The length of my argument is 16.
Hi! My thread index is 9 and my block index is 1. The length of my argument is 16.
Hi! My thread index is 10 and my block index is 1. The length of my argument is 16.
Hi! My thread index is 11 and my block index is 1. The length of my argument is 16.
Hi! My thread index is 12 and my block index is 1. The length of my argument is 16.
H

### Specifying type constraints in CUDA kernel arguments

From the point of view of a CUDA kernel, a `CuArray{T}` is a `CuDeviceArray{T}`. This is a new type that implements GPU-compatible operations, and the conversion is handled automatically for you. You can use multiple dispatch to define different kernels for different element types or array dimension and call the appropriate kernel.

In [ ]:
function my_first_cuda_kernel(a::CuDeviceArray{Float32})
    t_x = threadIdx().x
    b_x = blockIdx().x
    @cuprintln("Hi! My thread index is $t_x and my block index is $b_x. Wowow Float32 so speedy.")
    return
end

function my_first_cuda_kernel(a::CuDeviceArray{Float16})
    t_x = threadIdx().x
    b_x = blockIdx().x
    @cuprintln("Hi! My thread index is $t_x and my block index is $b_x. Float16 so slender.")
    return
end

my_first_cuda_kernel (generic function with 3 methods)

In [ ]:
CUDA.@sync @cuda threads=16 my_first_cuda_kernel(CUDA.rand(Float32, 16))

Hi! My thread index is 1 and my block index is 1. Wowow Float32 so speedy.
Hi! My thread index is 2 and my block index is 1. Wowow Float32 so speedy.
Hi! My thread index is 3 and my block index is 1. Wowow Float32 so speedy.
Hi! My thread index is 4 and my block index is 1. Wowow Float32 so speedy.
Hi! My thread index is 5 and my block index is 1. Wowow Float32 so speedy.
Hi! My thread index is 6 and my block index is 1. Wowow Float32 so speedy.
Hi! My thread index is 7 and my block index is 1. Wowow Float32 so speedy.
Hi! My thread index is 8 and my block index is 1. Wowow Float32 so speedy.
Hi! My thread index is 9 and my block index is 1. Wowow Float32 so speedy.
Hi! My thread index is 10 and my block index is 1. Wowow Float32 so speedy.
Hi! My thread index is 11 and my block index is 1. Wowow Float32 so speedy.
Hi! My thread index is 12 and my block index is 1. Wowow Float32 so speedy.
Hi! My thread index is 13 and my block index is 1. Wowow Float32 so speedy.
Hi! My thread index i

CUDA.HostKernel for my_first_cuda_kernel(CuDeviceVector{Float32, 1})

In [ ]:
CUDA.@sync @cuda threads=16 my_first_cuda_kernel(CUDA.rand(Float16, 16))

Hi! My thread index is 1 and my block index is 1. Float16 so slender.
Hi! My thread index is 2 and my block index is 1. Float16 so slender.
Hi! My thread index is 3 and my block index is 1. Float16 so slender.
Hi! My thread index is 4 and my block index is 1. Float16 so slender.
Hi! My thread index is 5 and my block index is 1. Float16 so slender.
Hi! My thread index is 6 and my block index is 1. Float16 so slender.
Hi! My thread index is 7 and my block index is 1. Float16 so slender.
Hi! My thread index is 8 and my block index is 1. Float16 so slender.
Hi! My thread index is 9 and my block index is 1. Float16 so slender.
Hi! My thread index is 10 and my block index is 1. Float16 so slender.
Hi! My thread index is 11 and my block index is 1. Float16 so slender.
Hi! My thread index is 12 and my block index is 1. Float16 so slender.
Hi! My thread index is 13 and my block index is 1. Float16 so slender.
Hi! My thread index is 14 and my block index is 1. Float16 so slender.
Hi! My thread i

CUDA.HostKernel for my_first_cuda_kernel(CuDeviceVector{Float16, 1})

## Avoiding warp divergence

As mentioned above, threads in the same warp executing different instructions forces each thread to execute in serial, dramatically slowing execution. So does this mean we can never use `if` or `else`? Let's do some experiments to find out.

In [ ]:
function diverging_kernel(arr)
    my_flag = arr[threadIdx().x]
    a = 1.0
    n_iter = my_flag > 0 ? (my_flag + 1)*5 : 20
    for ix in 1:n_iter
        if my_flag > 0
            a *= -2 * sin(rand() * π/4)
        else
            a += sqrt(0.25)
        end
    end
    return
end

diverging_kernel (generic function with 1 method)

Let's try launching and timing this with various values for `arr`.

In [ ]:
@cuda threads=64 diverging_kernel(CUDA.zeros(Int, 64));

In [ ]:
cu_arr = CuArray(rand(0:1, 64))
CUDA.@profile @cuda threads=64 blocks=50_000 diverging_kernel(cu_arr)

Profiler ran for 4.86 ms, capturing 12 events.

Host-side activity: calling CUDA APIs took 31.47 µs (0.65% of the trace)
┌──────────┬────────────┬───────┬────────────────┐
│ Time (%) │ Total time │ Calls │ Name           │
├──────────┼────────────┼───────┼────────────────┤
│    0.63% │   30.52 µs │     1 │ cuLaunchKernel │
└──────────┴────────────┴───────┴────────────────┘

Device-side activity: GPU was busy for 4.76 ms (97.84% of the trace)
┌──────────┬────────────┬───────┬──────────────────────────────────────────────┐
│ Time (%) │ Total time │ Calls │ Name                                         │
├──────────┼────────────┼───────┼──────────────────────────────────────────────┤
│   97.84% │    4.76 ms │     1 │ diverging_kernel(CuDeviceArray<Int64, 1, 1>) │
└──────────┴────────────┴───────┴──────────────────────────────────────────────┘


In [ ]:
cu_arr = CuArray(vcat(zeros(Int, 32), ones(Int, 32)))
CUDA.@profile @cuda threads=64 blocks=50_000 diverging_kernel(cu_arr)

Profiler ran for 2.49 ms, capturing 12 events.

Host-side activity: calling CUDA APIs took 31.71 µs (1.27% of the trace)
┌──────────┬────────────┬───────┬────────────────┐
│ Time (%) │ Total time │ Calls │ Name           │
├──────────┼────────────┼───────┼────────────────┤
│    1.20% │    29.8 µs │     1 │ cuLaunchKernel │
└──────────┴────────────┴───────┴────────────────┘

Device-side activity: GPU was busy for 2.39 ms (95.93% of the trace)
┌──────────┬────────────┬───────┬──────────────────────────────────────────────┐
│ Time (%) │ Total time │ Calls │ Name                                         │
├──────────┼────────────┼───────┼──────────────────────────────────────────────┤
│   95.93% │    2.39 ms │     1 │ diverging_kernel(CuDeviceArray<Int64, 1, 1>) │
└──────────┴────────────┴───────┴──────────────────────────────────────────────┘


In [ ]:
cu_arr = CUDA.ones(Int, 64)
CUDA.@profile @cuda threads=64 blocks=50_000 diverging_kernel(cu_arr)

Profiler ran for 4.85 ms, capturing 12 events.

Host-side activity: calling CUDA APIs took 31.23 µs (0.64% of the trace)
┌──────────┬────────────┬───────┬────────────────┐
│ Time (%) │ Total time │ Calls │ Name           │
├──────────┼────────────┼───────┼────────────────┤
│    0.61% │   29.33 µs │     1 │ cuLaunchKernel │
└──────────┴────────────┴───────┴────────────────┘

Device-side activity: GPU was busy for 4.75 ms (98.02% of the trace)
┌──────────┬────────────┬───────┬──────────────────────────────────────────────┐
│ Time (%) │ Total time │ Calls │ Name                                         │
├──────────┼────────────┼───────┼──────────────────────────────────────────────┤
│   98.02% │    4.75 ms │     1 │ diverging_kernel(CuDeviceArray<Int64, 1, 1>) │
└──────────┴────────────┴───────┴──────────────────────────────────────────────┘


In [ ]:
cu_arr = CuArray(rand(-1:3, 64))
CUDA.@profile @cuda threads=64 blocks=50_000 diverging_kernel(cu_arr)

Profiler ran for 9.6 ms, capturing 12 events.

Host-side activity: calling CUDA APIs took 30.28 µs (0.32% of the trace)
┌──────────┬────────────┬───────┬────────────────┐
│ Time (%) │ Total time │ Calls │ Name           │
├──────────┼────────────┼───────┼────────────────┤
│    0.30% │   28.61 µs │     1 │ cuLaunchKernel │
└──────────┴────────────┴───────┴────────────────┘

Device-side activity: GPU was busy for 9.5 ms (98.99% of the trace)
┌──────────┬────────────┬───────┬──────────────────────────────────────────────┐
│ Time (%) │ Total time │ Calls │ Name                                         │
├──────────┼────────────┼───────┼──────────────────────────────────────────────┤
│   98.99% │     9.5 ms │     1 │ diverging_kernel(CuDeviceArray<Int64, 1, 1>) │
└──────────┴────────────┴───────┴──────────────────────────────────────────────┘


### Forbidden operations

Not all valid Julia code is valid inside a CUDA kernel. In particular, within a kernel we **cannot use**:
- `String`s, except inside a `@cuprint`/`@cuprintln`/`@cuprintf` macro call
- `Exception`s
- Allocations (with two exceptions)

Additionally, type-unstable or uninferrable code may not be compilable and may generate long and sometimes cryptic errors.

## Working with memory inside kernels

When a warp executes, it has access to 3 types of memory:
- Global memory, or the on-device GPU RAM. This is the largest and slowest memory.
- Local memory, which holds locally defined variables for each thread.
- Shared memory, which is shared (wow, really?) among all the threads in the warp. This memory allows threads **to communicate with one another**.

Shared memory is physically located next to the cores executing the code and is fastest. To allocate shared memory, we can either define a `CuStaticSharedArray(T, N)`, with `N` elements of type `T` (`N` must be known at compile time) or a `CuDynamicSharedArray(T, N)`, where `N` isn't known at compile-time but will be passed to the kernel at *launch* time. **Note** that when you're using shared memory, you should call `sync_threads()` after any updates to it to make sure all threads in the warp can "see" the update (avoiding a data race).

In [ ]:
function reverse_kernel(a::CuDeviceArray{T}) where T
    i = threadIdx().x
    b = CuDynamicSharedArray(T, length(a))
    b[length(a)-i+1] = a[i]
    sync_threads()
    a[i] = b[i]
    return
end

cu_a = CuArray([1, 2, 3, 4, 5, 6, 7, 8])

@cuda threads=length(cu_a) shmem=sizeof(cu_a) reverse_kernel(cu_a)

CUDA.HostKernel for reverse_kernel(CuDeviceVector{Int64, 1})

The `shmem` argument should be the *total length* of shared memory needed across *all threads*. Many advanced GPU algorithms use shared memory for their effectiveness, so it's worth being aware of. The above example was pretty small -- do you think this would work if `a` were very large? Would we have to modify our kernel?

### Memory access patterns

As we discussed yesterday, the order in which we access array elements can have big performance impacts. In general, accessing adjacent memory locations in GPU memory is **much** faster than accessing locations that are widely separated. This is because to read a single location, the thread(s) have to load a whole "word" of memory at a time, so two memory locations that are part of the same word can be read cheaply once the word is loaded. This is also sometimes called "coalesced" memory access. It's most efficient to align warp-reads with 32-byte boundary memory regions -- something that using *strides* can often help with! How much of an impact can this have? For a matrix-matrix multiplication, NVIDIA ran tests on a V100 GPU. The observed memory bandwith was:

- Naive implementation: `12.8 GB/s`
- With shared memory to coalesce reads from global memory: `140.2 GB/s`
- Further optimizations from reduce bank conflicts: `199.4 GB/s`

Data taken from the [CUDA C++ programming guide](https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html#shared-memory). So, if your application involves a lot of reading and writing to memory, it's really worth figuring out how to use shared memory effectively. This is also something the profiler can help with.

## Benchmarking and profiling

`CUDA.jl` comes with a great integration with the NVIDIA CUDA profilers. This makes it easier to profile our kernels and find and fix performance problems. We can use the `CUDA.@profile` macro to see where we're spending our time. Let's revisit our threading example from above.

In [ ]:
n_arrs = 100
CUDA.@profile begin
    cpu_arrays = [rand(Float64, 1024, 1024) for ix_arr in 1:n_arrs]
    results = Vector{Float64}(undef, n_arrs)
    @sync begin
        for ix_arr in 1:n_arrs
            Threads.@spawn begin
                results[ix_arr] = mapreduce(sin, *, CuArray(cpu_arrays[ix_arr]))
            end
        end
    end
end

Profiler ran for 1.32 s, capturing 22681 events.

Host-side activity: calling CUDA APIs took 188.59 ms (14.27% of the trace)
┌──────────┬────────────┬───────┬───────────────────────────────────────┬─────────────────────────┐
│ Time (%) │ Total time │ Calls │ Time distribution                     │ Name                    │
├──────────┼────────────┼───────┼───────────────────────────────────────┼─────────────────────────┤
│   17.03% │  224.97 ms │   100 │   2.25 ms ± 0.92   (  1.83 ‥ 8.82)    │ cuMemcpyHtoDAsync       │
│    2.62% │   34.66 ms │   100 │ 346.64 µs ± 607.69 (  7.15 ‥ 2028.94) │ cuStreamCreate          │
│    0.78% │   10.37 ms │   300 │  34.55 µs ± 133.97 (  3.34 ‥ 2244.0)  │ cuMemAllocFromPoolAsync │
│    0.48% │    6.31 ms │   100 │  63.08 µs ± 252.41 ( 14.54 ‥ 1948.36) │ cuMemcpyDtoHAsync       │
│    0.27% │    3.51 ms │   200 │  17.57 µs ± 39.76  (  7.39 ‥ 566.01)  │ cuLaunchKernel          │
│    0.09% │    1.18 ms │   200 │   5.89 µs ± 3.56   (  2.38 ‥ 25.99)   │ c

We can also generate a trace to provide to NVIDIA's profile visualizer, which can give us lots of detailed information and tips about what to fix. We can either launch Julia [inside the `nsys` profiler](https://cuda.juliagpu.org/dev/development/profiling/#NVIDIA-Nsight-Systems) or generate the file by setting the `external=true` argument:

In [ ]:
n_arrs = 100
CUDA.@profile external=true begin
    cpu_arrays = [rand(Float64, 1024, 1024) for ix_arr in 1:n_arrs]
    results = Vector{Float64}(undef, n_arrs)
    @sync begin
        for ix_arr in 1:n_arrs
            Threads.@spawn begin
                results[ix_arr] = mapreduce(sin, *, CuArray(cpu_arrays[ix_arr]))
            end
        end
    end
end

You can then open the generated report file in NSight Systems and see a trace of what the GPU was doing when, how much load it was under, and use this information to improve your code (e.g. by overlapping kernel launches and memory copies). You can also use [NVIDIA NSight Compute](https://cuda.juliagpu.org/dev/development/profiling/#NVIDIA-Nsight-Compute) to do very detailed analysis on a single kernel and pinpoint things to fix.

**In GPU programming, you absolutely need to benchmark your kernels and use the profiler to fix any issues.**

## Some advantages of using Julia for GPU programming

- Easy to write kernels that are human-readable
- Can embed assembly in kernels to squeeze out more performance if necessary
- Good wrappers for most vendor libraries, including higher level APIs from `LinearAlgebra` and co.
- Seamless integration of CUDA streams and `Task`s to make scheduling multiple kernels ergonomic
- Good support for CUDA-aware MPI through `MPI.jl`
- Surprisingly easy to achieve good performance with a small amount of Julia code
- Relatively easy to implement stuff out of the sandbox that CUDA convenience libraries like `cuTile` provide
- Julia's package extension system makes providing optional GPU acceleration much more user-friendly
- A **lot** of Julia code "just works" on CUDA
- Multiple dispatch to kernels is very helpful and cuts down on code bloat

## Some issues remain
- Concrete eval doesn't work particularly well -- Julia's compiler can often evaluate `sin(1)` at compile time, but `CUDA.jl` wants to call the underlying NVIDIA `sin` assembly operation at runtime
- Not everything is wrapped (please open an issue!)
- We do wrap many low level primitives (such as for warp synchronization) but getting within 1% of C++ performance can be hard

## More resources

A lot of the tutorials and resources for CUDA and GPU programming in general are written in C or C++. Many of the concepts apply to writing performant kernels in Julia or any other language. Additionally, some older references don't account for new device features (like tensor cores) that NVIDIA has introduced over the years. The best technique is to implement something and profile it!

- [CUDA C Best Practices Guide](https://docs.nvidia.com/cuda/cuda-c-best-practices-guide)
- [JuliaCon 2021 GPU workshop](https://github.com/maleadt/juliacon21-gpu_workshop) -- 3 hours worth of tutorial content
- [`CUDA.jl` documentation](https://cuda.juliagpu.org/dev/)
- [`KernelAbstractions.jl`](https://juliagpu.github.io/KernelAbstractions.jl/dev/) -- write once, run on any GPU
- [`DaggerGPU.jl`](https://github.com/JuliaGPU/DaggerGPU.jl) -- graph-based scheduler that can take GPU resources into account
- [Hands on with Julia for HPC on GPUs and CPUs](https://www.youtube.com/watch?v=RNmSqbG2MUc)
- [Differentiable modeling on GPUs workshop](https://github.com/PTsolvers/gpu-workshop-JuliaCon23)
- [`#gpu` channel on Julia slack](https://julialang.org/slack/)
- [GPU category on Julia discourse](https://discourse.julialang.org/c/domain/gpu/11)
- [JuliaGPU Zoom office hours](https://julialang.org/community/), scroll down for the calendar invite

### Deep dive into GEMM kernel optimization

This pair of recent blog posts [(A)](https://jianyuh.github.io/gemm/optimization/hopper/2024/12/29/h100_gemm.html) and [(B)](https://cudaforfun.substack.com/p/outperforming-cublas-on-h100-a-worklog) discusses optimizing the GEMM kernel for H100 CUDA devices. It includes a deep dive into **multiple** steps of optimization, which we can quickly summarize here:

- Take advantage of the H100's tensor cores, which can compute matrix-matrix multiplication in a single instruction for small matrix sizes
- Implement **tiling** of outputs, and use larger output tiles which can help with memory access
- Hide *load* latency by interleaving tensor core ops and memory ops, which can be done with warp specialization
- Use special warpgroups to work with the output tile (prevents register spilling)
- Hide *store* latency by overlapping write operations for one thread block with load operations for another
- Force nearby tiles to be scheduled at the same time to use the L2 cache
- Use a device assembly instruction to implement a slightly faster barrier
- Group multiple thread blocks together into clusters to allow the kernel to load the same tile to multiple SMs
- A bunch of micro optimizations
- Using asynchronous stores to move data from the SM registers back into global memory
- Implement careful scheduling to improve L2 cache hit rate

Even with all this work, the optimized kernel is only **7%** faster than `CUBLAS` in the best case, while `CUBLAS` is **generally** quite performant. Partly this is because NVIDIA (and other vendors) employs an army of very smart people to write performant kernels. There are two main lessons from this:

1. Writing state of the art kernels is really hard
2. You should try to use the vendor library unless you have a good reason not to

## Using CUDA and other GPU libraries for QIS

CUDA provides several libraries besides `CUBLAS` which are useful for us as quantum information scientists:

- `CUTENSOR`, for tensor operations like permutation, contraction (Einstein summation), addition, etc.
- `CUSTATEVEC`, a GPU-accelerated library for state vector simulation of quantum circuits
- `CUTENSORNET`, a GPU-accelerated library for tensor network simulation of quantum circuits

The latter two are quite focused on quantum circuit simulation. In addition, many higher level packages provide these as backend simulation options. For example, `ITensors.jl` and the `TensorKit.jl` ecosystems both offer `CUTENSOR` accelerated backends.

Additionally, AMD offers their own `CUTENSOR` equivalent with an extremely similar API which is not yet wrapped by `AMDGPU.jl`.

[`Yao.jl`](https://github.com/QuantumBFS/Yao.jl), a circuit simulation library, also has an extension for CUDA acceleration for simulations.

In general, the higher level Julia libraries provide many convenience functions that the lower level CUDA libraries don't, and are portable to other HPC systems, so often the easier lift is to start with the high level package and file issues with `CUDA.jl` or `AMDGPU.jl` as needed to let maintainers (me & friends) fix things or add functionality.

## More about `KernelAbstractions.jl` and `AcceleratedKernels.jl`

As we said above, CUDA is probably the most mature GPU programming framework around, but AMD has put a lot of effort to improving their ROCm tooling for their GPUs, and Apple (Metal) and Intel (OneAPI) also offer GPU programming toolkits. Happily, Julia has a high level framework for targeting any of these backends, allowing you to write portable code that can run on various GPUs. This is particularly helpful if you have allocations at a variety of supercomputing centers, which tend to buy large sets of GPUs from either NVIDIA or AMD, and at which AMD is quite common.

`AcceleratedKernels.jl` works with `KernelAbstractions.jl` to provide a truly portable platform for writing "accelerated" code, **but**, as the authors state, "The API is starting to stabilise". So, many features and libraries (like sparse support) you may expect from `CUDA.jl` or `AMDGPU.jl` may not be supported yet (PRs welcome!). We can take a look at some sample kernels, again intended to run on NVIDIA hardware since that's what Google CoLab provides.

In [ ]:
Pkg.add("AcceleratedKernels"); Pkg.add("KernelAbstractions")
using AcceleratedKernels, KernelAbstractions

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
    Updating `~/.julia/environments/v1.10/Project.toml`
  [63c18a36] + KernelAbstractions v0.9.37
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [ ]:
m = CUDA.rand(Float32, 10, 100_000); # LARGE

In [ ]:
f(x) = x * x
mrowsumsq = AcceleratedKernels.mapreduce(f, +, m; init=zero(eltype(m)), dims=1)

1×100000 CuArray{Float32, 2, CUDA.DeviceMemory}:
 1.5963  4.19916  4.12753  2.98479  …  3.0629  4.2008  3.8935  3.30144

`AcceleratedKernels.jl` provides many "basic" array operations like `map`, `mapreduce`, `sort` -- you can do a lot of powerful things with these! For writing your own kernels, you can also do some light piracy and work to understand some of how these kernels are implemented under the hood:

In [ ]:
@kernel inbounds=true cpu=false unsafe_indices=true function _mapreduce_block!(@Const(src), dst, f, op, neutral)

    @uniform N = @groupsize()[1]
    sdata = @localmem eltype(dst) (N,)

    len = length(src)

    # NOTE: for many index calculations in this library, computation using zero-indexing leads to
    # fewer operations (also code is transpiled to CUDA / ROCm / oneAPI / Metal code which do zero
    # indexing). Internal calculations will be done using zero indexing except when actually
    # accessing memory. As with C, the lower bound is inclusive, the upper bound exclusive.

    # Group (block) and local (thread) indices
    iblock = @index(Group, Linear) - 0x1
    ithread = @index(Local, Linear) - 0x1

    i = ithread + iblock * (N * 0x2)
    if i >= len
        sdata[ithread + 0x1] = neutral
    elseif i + N >= len
        sdata[ithread + 0x1] = f(src[i + 0x1])
    else
        sdata[ithread + 0x1] = op(f(src[i + 0x1]), f(src[i + N + 0x1]))
    end

    @synchronize()

    if N >= 512u16
        if ithread < 256u16
            sdata[ithread + 0x1] = op(sdata[ithread + 0x1], sdata[ithread + 256u16 + 0x1])
        end
        @synchronize()
    end
    if N >= 256u16
        if ithread < 128u16
            sdata[ithread + 0x1] = op(sdata[ithread + 0x1], sdata[ithread + 128u16 + 0x1])
        end
        @synchronize()
    end
    if N >= 128u16
        if ithread < 64u16
            sdata[ithread + 0x1] = op(sdata[ithread + 0x1], sdata[ithread + 64u16 + 0x1])
        end
        @synchronize()
    end
    if N >= 64u16
        if ithread < 32u16
            sdata[ithread + 0x1] = op(sdata[ithread + 0x1], sdata[ithread + 32u16 + 0x1])
        end
        @synchronize()
    end
    if N >= 32u16
        if ithread < 16u16
            sdata[ithread + 0x1] = op(sdata[ithread + 0x1], sdata[ithread + 16u16 + 0x1])
        end
        @synchronize()
    end
    if N >= 16u16
        if ithread < 8u16
            sdata[ithread + 0x1] = op(sdata[ithread + 0x1], sdata[ithread + 8u16 + 0x1])
        end
        @synchronize()
    end
    if N >= 8u16
        if ithread < 4u16
            sdata[ithread + 0x1] = op(sdata[ithread + 0x1], sdata[ithread + 4u16 + 0x1])
        end
        @synchronize()
    end
    if N >= 4u16
        if ithread < 2u16
            sdata[ithread + 0x1] = op(sdata[ithread + 0x1], sdata[ithread + 2u16 + 0x1])
        end
        @synchronize()
    end
    if N >= 2u16
        if ithread < 1u16
            sdata[ithread + 0x1] = op(sdata[ithread + 0x1], sdata[ithread + 0x1 + 0x1])
        end
        @synchronize()
    end

    # Code below would work on NVidia GPUs with warp size of 32, but create race conditions and
    # return incorrect results on Intel Graphics. It would be useful to have a way to statically
    # query the warp size at compile time
    #
    # if ithread < 32
    #     N >= 64 && (sdata[ithread + 1] = op(sdata[ithread + 1], sdata[ithread + 32 + 1]))
    #     N >= 32 && (sdata[ithread + 1] = op(sdata[ithread + 1], sdata[ithread + 16 + 1]))
    #     N >= 16 && (sdata[ithread + 1] = op(sdata[ithread + 1], sdata[ithread + 8 + 1]))
    #     N >= 8 && (sdata[ithread + 1] = op(sdata[ithread + 1], sdata[ithread + 4 + 1]))
    #     N >= 4 && (sdata[ithread + 1] = op(sdata[ithread + 1], sdata[ithread + 2 + 1]))
    #     N >= 2 && (sdata[ithread + 1] = op(sdata[ithread + 1], sdata[ithread + 1 + 1]))
    # end

    if ithread == 0x0
        dst[iblock + 0x1] = sdata[0x1]
    end
end

_mapreduce_block! (generic function with 4 methods)

Wow! That is complicated and a big pain. However, this shows the amount of work that can go into writing a "top line" GPU kernel.

### Tuning

To get good performance out of a particular GPU for a particular use case, you may need to do (a **lot** of) "tuning". This involves benchmarking various usage scenarios -- more or less square matrix, "hot dog shaped" (technical term) matrix, triangular matrix, etc. -- on the real hardware and saving the results, which can then be used in a large decision tree to pick which kernel to launch. The tuning process can take many days or even weeks, and the results can be very large -- this is why some library shared object files are humongous (100s of MB). To perform tuning you will probably need a large amount of compute, so it is hard to do if you yourself aren't a GPU vendor or a supercomputing center.

## One more interesting package: `GemmKernels.jl`

A very common operation we'd like to perform on GPU is `GEMM` - general matrix multiplication. This operation is of extreme importance to ML/AI applications and GPU vendors and AI startups spend a lot of money to find people who can write good kernels for this. JuliaGPU has a package attemping to write some good open source GEMM kernels at `GemmKernels.jl` -- but the tuning process takes up to a week 🤯.